# Prepare the dependencies

In [ ]:
#!pip install --upgrade datasets[audio]

# Load libraries

In [121]:
from huggingface_hub import list_datasets
from datasets import(load_dataset, DatasetDict, load_from_disk,
                     Audio, IterableDatasetDict, Features, Value,
                     Sequence, Dataset)
import IPython.display as ipd
from functools import partial

# Load Dataset (Stream Method)

In [122]:
voice_data = load_dataset("malaysia-ai/malay-conversational-speech-corpus",
                          streaming=True, split="train", token=True)
voice_data

IterableDataset({
    features: ['Y', 'id', 'gender', 'filename'],
    n_shards: 1
})

In [123]:
def train_test_split(iterable_dataset, num_rows, split_ratio=0.8):
  taken = int(split_ratio * num_rows)
  print(f'taken: {taken}')
  raw_datasets = IterableDatasetDict()
  raw_datasets["train"] = iterable_dataset.take(taken)
  raw_datasets["test"] =  iterable_dataset.skip(taken)
  return raw_datasets

voice_data_split = train_test_split(voice_data, num_rows=3241, split_ratio=0.8)
voice_data_split

taken: 2592


IterableDatasetDict({
    train: IterableDataset({
        features: ['Y', 'id', 'gender', 'filename'],
        n_shards: 1
    })
    test: IterableDataset({
        features: ['Y', 'id', 'gender', 'filename'],
        n_shards: 1
    })
})

In [124]:
def pre_processing(voice_data):
  voice_data = voice_data.filter(lambda x: x["id"] != "0")
  voice_data = voice_data.rename_columns({"Y": "sentence", "filename": "audio"})
  voice_data = voice_data.remove_columns(["id", "gender"])
  voice_data = voice_data.cast(
    Features({'sentence': Value(dtype='string', id=None),
              'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None)
              }))
  voice_data = voice_data.cast_column("audio", Audio(sampling_rate=16000))

  return voice_data

In [125]:
def gen_from_iterable_dataset(iterable_ds):
    yield from iterable_ds

def convert_to_dataset(callable_fn, dataset_split):
  row_data = DatasetDict()
  row_data["train"] = Dataset.from_generator(
      partial(
          gen_from_iterable_dataset,
          callable_fn(dataset_split)["train"]),
      features=callable_fn(dataset_split)["train"].features)
  row_data["test"] = Dataset.from_generator(
      partial(
          gen_from_iterable_dataset,
          callable_fn(dataset_split)["test"]),
      features=callable_fn(dataset_split)["test"].features)
  return row_data

row_dataset = convert_to_dataset(pre_processing, voice_data_split)
row_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'audio'],
        num_rows: 2545
    })
    test: Dataset({
        features: ['sentence', 'audio'],
        num_rows: 604
    })
})

# Push to Hugging Face

In [93]:
row_dataset.push_to_hub('clt013/malay-speech-3k-rows-split')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/2545 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/26 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/604 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/clt013/malay-speech-3k-split/commit/be65007f84bb7877e1aabc63bf8d366710159c3f', commit_message='Upload dataset', commit_description='', oid='be65007f84bb7877e1aabc63bf8d366710159c3f', pr_url=None, pr_revision=None, pr_num=None)